In [2]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import models
from keras import layers
import warnings


Using TensorFlow backend.


In [3]:
#Cria o cabeçalho da lista CSV
cab = 'filename zero_cr chroma_cqt chroma_cens tonnetz chroma_stf rmse spec_centroid spec_bandwidth spec_contrast spec_rolloff'
for i in range(1, 21):
    cab += f' mfcc{i}'
cab += ' label'
cab = cab.split()


In [5]:
#Cria um CSV, localiza todas pastas e arquivos no diretório, extrai as features das músicas,
#concatena tudo dentro do CSV usando .append dentro do respetivo cabeçalho.
file = open('D:/Final/Dados em CSV/data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(cab)

generos = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() #Divide.
for g in generos: #Roda o for em todas pastas.
  
    for nome_arquivo in os.listdir(f'D:/Music Database/GTZAN Music Database/{g}'): #Roda arquivo por arquivo.
     
        musica = f'D:/Music Database/GTZAN Music Database/{g}/{nome_arquivo}'#Diretório da música.
     
        x, sr = librosa.load(musica, mono=True, duration=30) #Ler a música.
      
        #Extrai todas features abaixo, usando librosa.
        zcr = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
      
        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12, n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1
      
        chroma_cqt = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)

        chroma_cens = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)

        tonnetz = librosa.feature.tonnetz(librosa.effects.harmonic(x), sr=sr)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x
       
        chroma_stft = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)

        rmse = librosa.feature.rmse(S=stft)

        spec_cent = librosa.feature.spectral_centroid(S=stft)

        spec_bw = librosa.feature.spectral_bandwidth(S=stft)

        spec_contrast = librosa.feature.spectral_contrast(S=stft, n_bands=6)

        spec_rolloff = librosa.feature.spectral_rolloff(S=stft)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft

        mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)

        to_append = f'{nome_arquivo} {np.mean(zcr)} {np.mean(chroma_cqt)} {np.mean(chroma_cens)} {np.mean(tonnetz)} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(spec_contrast)} {np.mean(spec_rolloff)}'      
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('D:/Final/Dados em CSV/data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


In [6]:
#Lê o arquivo CSV usando o pandas.
todas_features = pd.read_csv('D:/Final/Dados em CSV/data.csv')
todas_features.head()


,filename,zero_cr,chroma_cqt,chroma_cens,tonnetz,chroma_stf,rmse,spec_centroid,spec_bandwidth,spec_contrast,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.083066,0.549554,0.273436,0.000684,0.349943,3.540670,1784.420446,2002.650192,20.530733,...,8.810668,-3.667367,5.751690,-5.162761,0.750947,-1.691937,-0.409954,-2.300208,1.219928,blues
1,blues.00001.wav,0.056044,0.503215,0.261366,0.018852,0.340983,2.601917,1529.835316,2038.617579,20.676334,...,5.376802,-2.239119,4.216963,-6.012273,0.936109,-0.716537,0.293875,-0.287431,0.531573,blues
2,blues.00002.wav,0.076301,0.515355,0.267732,0.022247,0.363603,4.798367,1552.481958,1747.165985,22.197265,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00003.wav,0.033309,0.400808,0.238640,0.008652,0.404779,3.852295,1070.119953,1596.333948,21.428764,...,6.087676,-2.476420,-1.073890,-2.874777,0.780976,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.101500,0.521289,0.266510,-0.050764,0.308590,2.487634,1835.494603,1748.362448,21.467739,...,-2.806385,-6.934122,-7.558619,-9.173552,-4.512166,-5.453538,-0.924162,-4.409333,-11.703781,blues


In [8]:
#Exclui a coluna com os nomes.
todas_features = todas_features.drop(['filename'],axis=1)


In [9]:
#Cria uma ligação entre os generos e um valor int, cada int representa um genero.
genre_list = todas_features.iloc[:, -1] #Cria um int para cada label
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list) #Separa por labens distintas


In [10]:
#X é calculado removendo o mean e divindo pela variancia.
scaler = StandardScaler()
X = scaler.fit_transform(np.array(todas_features.iloc[:, :-1], dtype = float))   


In [11]:
#Dividi os dados entre treino e teste em 800/200.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,test_size=0.2, random_state=101)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(800, 30) (200, 30) (800,) (200,)


In [12]:
#Cria o modelo da CNN, com 256 neurônios.
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [13]:
#Definições de como o modelo deverá ser treinado.
model.compile(optimizer='adam', #Algoritimo Adam para otimizar a rede neural.
              loss='sparse_categorical_crossentropy', #Função que utiliza as baixas.
              metrics=['accuracy']) #Precisão escolhida como métrica


In [18]:
#Treinando o modelo.
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=64)


Epoch 1/20
800/800 [==============================] - 0s 76us/step - loss: 0.0411 - acc: 0.9975
Epoch 2/20
800/800 [==============================] - 0s 64us/step - loss: 0.0396 - acc: 0.9975
Epoch 3/20
800/800 [==============================] - 0s 62us/step - loss: 0.0322 - acc: 0.9988
Epoch 4/20
800/800 [==============================] - 0s 45us/step - loss: 0.0331 - acc: 0.9962
Epoch 5/20
800/800 [==============================] - 0s 41us/step - loss: 0.0297 - acc: 0.9962
Epoch 6/20
800/800 [==============================] - 0s 45us/step - loss: 0.0262 - acc: 0.9988
Epoch 7/20
800/800 [==============================] - 0s 42us/step - loss: 0.0244 - acc: 0.9988
Epoch 8/20
800/800 [==============================] - 0s 44us/step - loss: 0.0227 - acc: 0.9975
Epoch 9/20
800/800 [==============================] - 0s 46us/step - loss: 0.0229 - acc: 0.9975
Epoch 10/20
800/800 [==============================] - 0s 45us/step - loss: 0.0193 - acc: 0.9975
Epoch 11/20
800/800 [==================

In [19]:
#Calculando a precisão e perca que o modelo conseguiu categorizando da musica baseado nas features extraidas.
test_loss, test_acc = model.evaluate(X_test,y_test)
print('Precisão: ', test_acc)
print('Perdido: ', test_loss)


200/200 [==============================] - 0s 45us/step
Precisão:  0.655
Perdido:  1.9735085678100586


In [20]:
#Separando 200 amostras para o teste de validação.
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]


In [21]:
#Cria o modelo da CNN, com 512 neurônios.
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [22]:
#Definições de como o modelo deverá ser treinado.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Treinando o modelo.
model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=256,
          validation_data=(x_val, y_val))


Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 1s 2ms/step - loss: 2.2601 - acc: 0.1950 - val_loss: 2.0842 - val_acc: 0.2700
Epoch 2/30
600/600 [==============================] - 0s 57us/step - loss: 2.0102 - acc: 0.2850 - val_loss: 1.8770 - val_acc: 0.2950
Epoch 3/30
600/600 [==============================] - 0s 53us/step - loss: 1.7925 - acc: 0.3067 - val_loss: 1.6971 - val_acc: 0.3600
Epoch 4/30
600/600 [==============================] - 0s 53us/step - loss: 1.6037 - acc: 0.4050 - val_loss: 1.5417 - val_acc: 0.5000
Epoch 5/30
600/600 [==============================] - 0s 52us/step - loss: 1.4481 - acc: 0.5167 - val_loss: 1.3928 - val_acc: 0.5200
Epoch 6/30
600/600 [==============================] - 0s 52us/step - loss: 1.3138 - acc: 0.5383 - val_loss: 1.3268 - val_acc: 0.5550
Epoch 7/30
600/600 [==============================] - 0s 50us/step - loss: 1.1805 - acc: 0.6250 - val_loss: 1.2403 - val_acc: 0.5750
Epoch 8/30
600/600 [====

In [23]:
resultado = model.evaluate(X_test, y_test)
resultado


200/200 [==============================] - 0s 324us/step


[1.3783106994628906, 0.67]